In this Notebook, we will be exploring how to create a `csv_writer()` function.

The function should accomplish a single task: When given a list of game data and a filename, it should create a `.csv` file of the provided name containing easily-parsable game data.

We have 2 possible approaches to this:
* A generic function that will work regardless of the game data provided
* Unique functions for each game

Both approaches are valid, but come with their own benefits and drawbacks.

Creating a generic function would allow us to write the least amount of code. It would also encourage a standarized format for data files, as each game would be written in by the same function in the same format (In reality, this can be either a benefit or a drawback).

Creating unique functions for each game would require more up-front coding, but coule reduce ambiguity and problems down the road related to "overly-abstract" code. Additionally, it would give us the flexability needed to deviate from a standard `.csv` format if needed.

Speaking of format, let's discuss what kind of format our data should be presented in.

Again, we find ourselves with two approaches:

| option 1 |        state_1       | state_2              | ... |       state_k        | winner     |
|:--------:|:--------------------:|----------------------|:---:|----------------------|------------|
|  game_1  | playername move_made | playername move_made | ... | playername move_made | playername |
|  game_2  | playername move_made | playername move_made | ... | playername move_made | playername |
|    ...   |          ...         | ...                  | ... |         ...          |    ...     |
|  game_n  | playername move_made | playername move_made | ... | playername move_made | playername |

| option 2 | state_1 PLAYER | state_1 MOVE | state_2 PLAYER | state_2 MOVE | ... | state_k PLAYER | state_k MOVE | winner     |
|:--------:|:--------------:|--------------|----------------|--------------|:---:|----------------|--------------|------------|
|  game_1  |   playername   | move_made    | playername     | move_made    | ... | playername     | move_made    | playername |
|  game_2  |   playername   | move_made    | playername     | move_made    | ... | playername     | move_made    | playername |
|    ...   |       ...      | ...          | ...            | ...          | ... | ...            |  ...         |   ...      |
|  game_n  |   playername   | move_made    | playername     | move_made    | ... | playername     | move_made    | playername |

Let us first consider the similarities:
* An entire game's data fits into a single row
* The columns represent the game states
* Entries in the table contain the move made at that state in the game and the player who made that move

Now let us acknowledge the differences:
* Option 1:
    * Columns are game states
    * Each table entry is a pair, containing BOTH the player who moved and the move made
* Option 2:
    * There are twice as many columns
    * Columns are "paired" with each pair being the player whose turn it was at that game state and the move made at that game state
    * Each entry is *either* a player name *or* a move made

Both options provide decent benefits and drawbacks. Let us observe them.

Option 1 provides the most concise way of viewing the data. Rows and columns cleanly provide ways to access the necessary data. However, extracting bits of data from the table may cause problems. For example, if one wishes to read the `.csv` into a python DataFrame, additional parsing would be needed to extra *just* the "move_made" or *just* the "playername" at a given board state. Additionally, special care would need to go into deciding a delimiter to separate the player name and move made. Obviously, it cannot be a comma. The pipe symbol `|` may work. Finally, this may require less code initially as the `(playername, move_made)` format is how the game data is stored in the `board.data` object.

Option 2 provides a much more complex way of viewing the data. To extrapolate important information, one needs to isolate a column pair. Additional code would be needed to identify column pairs based on game state. However, extracting bits of data may be significantly easier. Indexing directly into a row/column pair will give the immediate data at that entry, without the need to parse additionally.

We do not initially know which format will be easiest to work with. We know that both will require additional code/work to extrapolate specific bits of data. We will investigate both formats.

Obviously, we start by importing everything we need to run some games.

In [24]:
from classes.player import Player
from classes.board import Board
from classes.referee import Referee
from classes.game import Game

# Import the rulesets and strategy
from toothpick_takeaway.classes.toothpick_ruleset import ToothpickRuleset
from toothpick_takeaway.classes.toothpick_strategy import ToothpickStrategy
from unbalanced_rook.classes.rook_ruleset import RookRuleset
from unbalanced_rook.classes.rook_strategy import RookStrategy

Next let's set up the game(s).
As usual, this consists of:
* Initializing a ruleset and strategy
* Initializing two players
* Initializing a board
* Initializing a referee
* Initializing a game

Then we run the games in a loop, appending the returned data to a list to be used later.

Comment out either the Toothpick ruleset/strategy code, or the Rook ruleset/strategy code, depending on what game you want to run.

In [2]:
#ruleset =  ToothpickRuleset("Toothpick Takeaway Ruleset", initial_state=10, bounds=2)
#strategy = ToothpickStrategy("Random Strategy for Toothpick Takeaway")
ruleset =  RookRuleset("Rook Takeaway Ruleset", initial_state={"D": 0, "R": 0}, bounds={"D": 10, "R": 10})
strategy = RookStrategy("Random Strategy for Unbalanced Rook")

p1 = Player("player1", strategy)
p2 = Player("player2", strategy)

board = Board(ruleset.initial_state, ruleset.bounds)

ref = Referee(board, ruleset)

game = Game(ref, board, [p1, p2])

games = []
for i in range(10):

    data = game.play().data
    games.append(data)

As a reference, here is the `csv_writer` function we used for Toothpick Takeaway last Fall:

In [25]:
import csv
def write_to_csv(game_data, filename):
    """
    Writes game data to a .csv file.

    Parameters:
        game_data (list): Data of each turn made in the game and the winner of that game
        filename (string): Name of the file to write to
    """
    # Get the total number of toothpicks at the start
    start_val = max(game_data[0].history.keys())

    # Make a descending list of all toothpicks left
    toothpicks_left = list(range(start_val, 0, -1))

    # Create our headings: Toothpicks left and turn
    headings = []
    for heading in toothpicks_left:
        headings.append(heading)
        headings.append("turn_{}".format(heading))
    headings.append("winner")

    # Start building rows; one row per game
    rows = []
    for summary in game_data:
        # How many toothpicks were taken at each turn in the game
        turns = [summary.history[turn]["move"] for turn in summary.history]
        # Who took those toothpicks
        names = [summary.history[turn]["name"] for turn in summary.history]

        # Start creating a row
        moves = []
        for i in range(len(turns)):
            # Append the toothpicks taken
            moves.append(turns[i])
            # Append the player who took them
            moves.append(names[i])

            # If a turn was 2, add a turn of 0 after it
            # This ensures that our rows are all the same length
            if turns[i] == 2:
                moves.append(None)
                moves.append(None)

        # Append the winner of the game
        moves.append(summary.winner)

        # Add the row we just made to the running list of rows
        rows.append(moves)

    # Write to csv
    with open(filename, "w") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(headings)
        csvwriter.writerows(rows)

The code is not elegant, generic, or streamlined. It follows the format described in Option 2 above. It serves as an excellent reference.

Now let's begin writing a basic `csv_writer` function. For this trial, we will attempt a generic csv writer that follows Option 1's format from above.

In [26]:
import csv

def csv_writer(game_data, filename, delimiter = "|"):
    """
    Generic CSV writer. Writes game data to a .csv file.

    Parameters:
        game_data (list): Data of each turn made in the game and the winner of that game
        filename (string): Name of the file to write to
        delimiter (string): The character between "playername" and "move made"
    """
    # Step 1: Get a list of all possible game states
    # These server as the headers or column names in the csv
    # In this example, we iterate over the game data provided
    # and extract all unique game states into a list
    # Then, we sort the list (just to make it a bit more readable)
    game_states = [] 
    for data in game_data:
        # "data" is a single game's history

        for state in data.keys():
            if state not in game_states:
                game_states.append(str(state))
    game_states.sort()
    
    # Now we need to build each row of the csv
    rows = []
    for data in game_data:
        # "data" is a single game's history
        # A single row represents a single game's move history
        row = []
        for state in data.keys():
            pass

Let's test out our function to see the results it produces:

In [23]:
# Toothpick Game State

csv_writer(games, "test_file")

### Notes

Writing to a `.csv` format takes a significant amount of planning to ensure a good format.

Here are some important things to consider when writing the function:
* All possible game states *must* be known when creating the csv. This is because the columns of the table must represent the states of the game at every turn.
* The columns do not necessarily *need* to be in order. Ordering them would certainly make the game data easier to read, but it is not a requirement
    * "In order" will also differ depending on the game. For Toothpick Takeaway, "ordered" means descending from the max number of toothpicks to 0, whereas in Unbalanced Rook, "ordered" means `(0,0)` to the end of the board, `(n,m)`, in the sequence they were made.
* Once all possible board states are known, a list of these states should be created to represent the "headings" or "columns" of the `.csv`, with the addition of a "winner" column at the end.
* We now need to create a `row`, (list) for each game
* Each `row` needs to contain:
    * The moves made at every state in the game
    * `None` when no move was made at the game state in the current column
    * The winner's name, at the end

If that last bullet seems confusing, it's because it is. There are multiple ways of doing this, but none seem efficient or wise.
Essentially, we need to iterate over the list of all possible game states and then, for every game, if the game contains a move made at that game state, we store the move, otherwise we store `None` and continue iterating. This seems terribly inefficient.